In [1]:
import nltk.stem
import re
import numpy as np
import os
import nltk
import pandas as pd
from src.SemEvalData import SemEvalData
from src.JigsawData import JigsawData
from src.preprocessing import preprocess_bayes, getSpansByToxicWords, getToxicWordsBayes
import joblib
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets
import sklearn.feature_extraction.text
import sklearn.naive_bayes
import sklearn.metrics
import sklearn.model_selection
import sklearn.pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from test_sentence import test_bayes

In [2]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Julia\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
#stemmer = nltk.stem.SnowballStemmer('english')
lemmer = nltk.stem.WordNetLemmatizer()

In [4]:
## load data
train_data_semeval = SemEvalData()
train_data_semeval.load_data("data/tsd_train.csv")
train_df_preprocessed = train_data_semeval.preprocess()

In [7]:
train_df_preprocessed.head(25)

,spans,text,toxicity,toxic_words,original_text,sentences,diff,toxicity_sentence
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",another violent and aggressive immigrant killi...,1,[violent and aggressive immigrant],Another violent and aggressive immigrant killi...,[another violent and aggressive immigrant kill...,[],[1.0]
1,"[33, 34, 35, 36, 37, 38, 39]","i am 56 years old, i am not your fucking junio...",1,[fucking],"I am 56 years old, I am not your fucking junio...","[i am 56 years old, i am not your fucking juni...","[94, 575, 588, 647]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,"[0, 1, 2, 3]","damn, a whole family. sad indeed.",1,[damn],"Damn, a whole family. Sad indeed.","[damn, a whole family., sad indeed.]",[],"[1.0, 0.0]"
3,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]",what a knucklehead. how can anyone not know th...,1,[knucklehead],What a knucklehead. How can anyone not know th...,"[what a knucklehead., how can anyone not know ...",[],"[1.0, 0.0, 0.0]"
4,"[32, 33, 34, 35, 36, 37, 38]",who do you think should do the killing? anyon...,1,[killing],"""who do you think should do the killing?""\n\nA...","[who do you think should do the killing?, anyo...","[0, 40, 442, 622, 933]","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
5,[],"but, but, but, is not a defense. its not even...",0,[],"But, but, but, is NOT a defense. It's not eve...","[but, but, but, is not a defense., its not eve...","[36, 250, 276]","[0.0, 0.0, 0.0, 0.0, 0.0]"
6,"[39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 5...","please people, stop using these silly, stupid ...",1,"[stupid, emoticons]","Please people, stop using these silly, stupid ...","[please people, stop using these silly, stupid...",[],[1.0]
7,"[0, 1, 2, 3]",dumb.,1,[dumb],Dumb.,[dumb.],[],[1.0]
8,"[49, 50, 51, 52, 53, 54, 147, 148, 149, 150, 1...",obamacare is on its last gasping breaths. yo...,1,"[idiots, stupid]",Obamacare is on it's last gasping breaths. Y...,"[obamacare is on its last gasping breaths., yo...","[18, 63, 97, 227]","[0.0, 1.0, 0.0, 0.0]"
9,"[32, 33, 34, 35, 36, 37, 38, 39]",crooked trump = guilty as hell. pathetic,1,[pathetic],CROOKED Trump = GUILTY as hell.\npathetic,"[crooked trump = guilty as hell., pathetic]",[],"[0.0, 1.0]"


In [8]:
train_data = {
    'sentence':  train_df_preprocessed.sentences.sum(),
    'toxicity_sentence': train_df_preprocessed.toxicity_sentence.sum()
        }

train_df = pd.DataFrame (train_data, columns = ['sentence','toxicity_sentence'])

In [9]:

train_df['sentence'] = train_df.apply(lambda row: preprocess_bayes(row.sentence), axis=1)
print(train_df.head(5))

                                            sentence  toxicity_sentence
0  another violent and aggressive immigrant killi...                1.0
1   i am  years old i am not your fucking junior pal                1.0
2                 what you are saying makes no sense                0.0
3            i dont know what you are basing this on                0.0
4  the cheap black market crap is still coming up...                0.0


In [10]:
len(train_df[train_df["toxicity_sentence"] == 0])

13264

In [11]:
len(train_df[train_df["toxicity_sentence"] == 1])

9500

In [12]:
def stemming(text):
    # Stem words
    #data[i] = ' '.join([stemmer.stem(word) for word in data[i].split()])
    data = ' '.join([lemmer.lemmatize(word) for word in text.split()])
    return data


In [13]:
# Perform a grid search to find the best hyperparameters
def grid_search(train):
    # Create a pipeline
    clf_pipeline = sklearn.pipeline.Pipeline([
        ('v', CountVectorizer(strip_accents='ascii', stop_words='english')),
        ('t', TfidfTransformer()), 
        ('c', sklearn.naive_bayes.MultinomialNB(fit_prior=True, class_prior=None))
        ])
    # Set parameters (name in pipeline + name of parameter)
    parameters = { 
        'v__ngram_range': [(1, 1), (1, 2), (1, 3), (1, 4)], 
        'v__lowercase': (True, False), 
        't__use_idf': (True, False), 
        'c__alpha': (0.3, 0.6, 1.0) }
    # Create a grid search classifier
    gs_classifier = sklearn.model_selection.GridSearchCV(clf_pipeline, parameters, cv=5, iid=False, n_jobs=2, scoring='accuracy', verbose=1)
    
    # Start a search (Warning: takes a long time if the whole dataset is used)
    # Slice: (train.data[:4000], train.target[:4000])
    gs_classifier = gs_classifier.fit(train.data, train.target)
    # Print results
    print('---- Results ----')
    print('Best score: ' + str(gs_classifier.best_score_))
    for name in sorted(parameters.keys()):
        print('{0}: {1}'.format(name, gs_classifier.best_params_[name]))

In [14]:
train_df['sentence'] = train_df.apply(lambda row: stemming(row.sentence), axis=1)

In [15]:
print(train_df.head())

                                            sentence  toxicity_sentence
0  another violent and aggressive immigrant killi...                1.0
1     i am year old i am not your fucking junior pal                1.0
2                  what you are saying make no sense                0.0
3            i dont know what you are basing this on                0.0
4  the cheap black market crap is still coming up...                0.0


In [16]:
# Train and evaluate a model
def train_and_evaluate(train):
    
    # Convert to bag of words
    count_vect = CountVectorizer(strip_accents='ascii', stop_words='english', lowercase=True, ngram_range=(1,1))
    X = count_vect.fit_transform(train['sentence'])
    # Convert from occurrences to frequencies
    # Occurrence count is a good start but there is an issue: longer documents will have higher average count values than shorter documents, even though they might talk about the same topics.
    # To avoid these potential discrepancies it suffices to divide the number of occurrences of each word in a document by the total number of words in the document: these new features are called tf for Term Frequencies.
    transformer = TfidfTransformer()
    X = transformer.fit_transform(X)
    # Create a model
    # get the first vector out (for the first document) 
    first_vector_tfidfvectorizer=X[0] 
 
    # place tf-idf values in a pandas data frame 
    df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=count_vect.get_feature_names(), columns=["tfidf"]) 
    df.sort_values(by=["tfidf"],ascending=True)
    print(df.head)
    model = sklearn.naive_bayes.MultinomialNB(alpha=0.3, fit_prior=True, class_prior=None)
    # Train the model
    model.fit(X, train['toxicity_sentence'])
    # Save models
    joblib.dump(count_vect, 'vectorizer_bayes.jbl')
    joblib.dump(transformer, 'transformer_bayes.jbl')
    joblib.dump(model, 'model_bayes.jbl')
    # Evaluate on training data
    print('-- Training data --')
    predictions = model.predict(X)
    print('here',model.coef_, len(count_vect.get_feature_names()), predictions)
#     neg_class_prob_sorted = model.feature_log_prob_[0, :].argsort()
#     pos_class_prob_sorted = model.feature_log_prob_[1, :].argsort()

#     print(np.take(count_vect.get_feature_names(), neg_class_prob_sorted[:10]))
#     print(np.take(count_vect.get_feature_names(), pos_class_prob_sorted[:10]))
    print(predictions,X, train['sentence'])
    accuracy = sklearn.metrics.accuracy_score(train['toxicity_sentence'], predictions)
    print('Accuracy: {0:.2f}'.format(accuracy * 100.0))
    print('Classification Report:')
    print(sklearn.metrics.classification_report(train['toxicity_sentence'], predictions))
    print('')
#     Evaluate with 10-fold CV
    print('-- 10-fold CV --')
    predictions = sklearn.model_selection.cross_val_predict(model, X, train['toxicity_sentence'], cv=10)
    accuracy = sklearn.metrics.accuracy_score(train['toxicity_sentence'], predictions)
    print('Accuracy: {0:.2f}'.format(accuracy * 100.0))
    print('Classification Report:')
    print(sklearn.metrics.classification_report(train['toxicity_sentence'], predictions))
    return model


In [17]:

train_and_evaluate(train_df)

<bound method NDFrame.head of          tfidf
aa         0.0
aaa        0.0
aahhh      0.0
aapoor     0.0
abandon    0.0
...        ...
zuptoid    0.0
zuri       0.0
zwei       0.0
zz         0.0
zzzzzz     0.0

[17812 rows x 1 columns]>
-- Training data --
here [[ -8.99365183 -10.46043916 -10.3795395  ... -11.42007242 -11.42007242
  -11.42007242]] 17812 [1. 1. 0. ... 0. 0. 1.]
[1. 1. 0. ... 0. 0. 1.]   (0, 17025)	0.3612465882102366
  (0, 13766)	0.4250020107598058
  (0, 8722)	0.2989512824534744
  (0, 8171)	0.349476259432824
  (0, 8079)	0.3323369916456108
  (0, 7760)	0.3248117045584413
  (0, 2604)	0.3030879853045209
  (0, 320)	0.41177156518958624
  (1, 17689)	0.31672999210173375
  (1, 11354)	0.4934807414263961
  (1, 10986)	0.3800480289834355
  (1, 8568)	0.5424937095001565
  (1, 6174)	0.4662863291616598
  (2, 14090)	0.658914520992524
  (2, 13808)	0.5930647728437727
  (2, 9514)	0.46271571103319686
  (3, 8797)	0.3879256621830136
  (3, 4524)	0.35660724746419836
  (3, 1290)	0.8499087902097956

MultinomialNB(alpha=0.3)

In [16]:
##load from file
count_vect = joblib.load('vectorizer.jbl')
transformer = joblib.load('transformer.jbl')
model = joblib.load('model.jbl')

In [18]:
train_data_semeval = SemEvalData()
train_data_semeval.load_data("data/tsd_train.csv")
train_df_preprocessed = train_data_semeval.preprocess()

In [ ]:
train_df_preprocessed["predicted_span"]=[test_bayes(sentences)for sentences in train_df_preprocessed["original_text"] ]

In [17]:
samples_to_test = train_df.head(19)
for sample in samples_to_test.itertuples():
    print(sample)
    x = count_vect.transform([sample.sentence])
    y = model.predict(x)
    if y == 1.0:
        toxic = getToxicWordsBayes(count_vect, x, 0.5)
        spans = getSpansByToxicWords(toxic,sample.sentence)

Pandas(Index=0, sentence='another violent and aggressive immigrant killing a innocent and intelligent u citizen sarcasm', toxicity_sentence=1.0)
['aggressive', 'citizen', 'immigrant', 'innocent', 'intelligent', 'killing', 'sarcasm', 'violent'] another violent and aggressive immigrant killing a innocent and intelligent u citizen sarcasm []
Pandas(Index=1, sentence='i am year old i am not your fucking junior pal', toxicity_sentence=1.0)
['fucking', 'junior', 'old', 'pal', 'year'] i am year old i am not your fucking junior pal []
Pandas(Index=2, sentence='what you are saying make no sense', toxicity_sentence=0.0)
Pandas(Index=3, sentence='i dont know what you are basing this on', toxicity_sentence=0.0)
Pandas(Index=4, sentence='the cheap black market crap is still coming up from mexican national and the good stuff from bc and humbolt', toxicity_sentence=0.0)
Pandas(Index=5, sentence='the i corridor is still just a busy and they are still making a ton of money', toxicity_sentence=0.0)
Pand

## Evaluation

In [18]:
test_data_semeval = SemEvalData()
test_data_semeval.load_data("data/tsd_trial.csv")
test_df_preprocessed = test_data_semeval.preprocess()

In [19]:
test_df_preprocessed["predicted_span"]=[test_bayes(sentences)for sentences in test_df_preprocessed["original_text"] ]

['bigot', 'moron'] because hes a moron and a bigot its not any more complicated than that [10, 32, 36, 73]
['add', 'bleach', 'gene', 'idiot', 'let', 'nature', 'pool', 'protecting', 'stop'] how about we stop protecting idiots and let nature add some bleach to the gene pool we can always submit their names for the darwin awards [83, 139]
[] if people  were  smart they would  boycott this  inept  airline  but   they  are  not  smart  so   rogue  businesses  like  this  one   still thrive   taking the idiots  for  a ride [22, 64, 94, 136, 185, 186, 187]
[] trump claimed that russia will never invade the ukraine when russia already has  how stupid can people be [55, 81, 107]
['hypocritical', 'wanker'] as long as your willing to pay a lot more for products you buy then fine but you better not be going to costco and walmart to buy stuff because its cheaper if so we get to call you a hypocritical wanker [62, 73, 148, 158, 165, 207]
['believe', 'idiot', 'machine', 'propaganda', 'publishes', 're

['court', 'death', 'drug', 'druggies', 'duterte', 'enemy', 'killed', 'killing', 'political', 'right', 'sentenced', 'taken', 'unless', 'unpunished', 'use', 'case', 'countryman', 'dictator', 'fascist', 'turkey'] people are tired of seeing their countries overrun with illegal immigrants crime  social parasites and welfare refugees and i applaud hungary eastern europe to protect their citizens and approve of  lepen duterte is right killing druggies  but should be taken to court and sentenced to death if he had political enemies killed  unless they favored unpunished drug use  and turkey is a case of islamism and a fascist dictator persecuting his own countrymen [74, 82, 143, 207, 243, 332, 373, 375, 463]
['dumb', 'forever', 'maybe', 'stupid'] trying the education route is best but you face a huge problem in that the average person today is dumber than a doughnut  stupid maybe you can do something but dumb dumb is forever [121, 130, 167, 184]
['couple', 'cute', 'idiot', 'make'] maybe this a

['bull', 'shit', 'bullshit', 'word'] bull shit did i spell that right or is it one word bullshit jeff bezos do you tremble at the prospect of president clinton [9, 33, 52, 62, 74, 127]
['create', 'hatred', 'islam', 'muslim', 'say', 'terrorist', 'used', 'area', 'deep', 'gray', 'stupidity', 'wide'] so if i were to say that islam is being used to create terrorists which it is would that be hatred towards muslims  the gray area on this stupidity is wide and deep [65, 78, 116, 167]
['cabinet', 'racist', 'scum', 'trudeau', 'ash', 'throw'] trudeau and cabinet are the racist scum throw them all into the volcano ashes to ashes [39, 72, 88]
['comment', 'dangerous', 'reckless', 'stupid', 'trump'] trumps comments are stupid reckless and very dangerous he is temperamentally and intellectually unfit to be the person responsible for the lives of millions [5, 27, 56, 159]
['alaska', 'girl', 'kick'] alaska girls kick ass [21]
['freaking', 'idiot'] how about disturbing the peace  hope he gets at least t

['agent', 'manipulative', 'narcissist', 'breakfast', 'cheat', 'dinner', 'lie', 'lunch'] re agents are some of the most manipulative narcissists around  they lie lie lie cheat cheat cheat for breakfast lunch and dinner   how do you teach someone to sell their soul [54, 63, 74, 79, 84, 91, 98, 119, 137, 140, 185]
['leftist', 'demonize', 'help', 'just', 'need', 'people', 'propaganda', 'remark', 'silly', 'thats', 'bunch', 'hypocrite'] thats contradicted by the facts first of all democrats have acknowledged the past and the party evolved to the antiracist side which prompted southern racists who used to constitute the solid south for the party to migrate to the gop where the south now pretty much votes solidly for republicans due to the gops adoption of the racist polices the democrats used to push if there are any remaining racists in the democratic party the members will marginalize and push them out  the democratic party of the s though was the conservative party they were not leftist at

['electoral', 'gaining', 'ignorant', 'major', 'misogynistic', 'racist', 'selfish', 'snowflake', 'trump', 'vote', 'fix', 'stupid'] ignorant selfish racist misogynistic snowflakes were the major part of trump gaining enough electoral votes  you cant fix stupid [8, 17, 25, 110, 120, 133]
['canadian', 'silly'] justin trudeau is too busy with his pay for access chinese friends and getting advice from and giving taxpayers money to the billionaire islamic spiritual leader the aga khan than worry about the fact that canadians cant afford to live in their own cities anymore   silly canadians [219, 264, 283]
['amish', 'bunch', 'clearly', 'focus', 'group', 'idiot', 'im', 'like', 'listed', 'mafia', 'network', 'sure', 'think', 'youve'] all you said is true however we as viewers arent the ones responsible for programming im not sure how shows like  the shows youve listed or amish mafia mermaids and the like fared in focus groups or who previewed these shows before airing but clearly the cable networ

...

[] close the msm down get rid of the jews that run themthey are the problem [18, 53, 54]
['stupid', 'thats'] thats stupid [4, 13]
['head', 'rear', 'russia', 'threat', 'ugly', 'democrat', 'idiotic', 'newspaper'] you mean not russia  you know since trump was elected  and even before he made a commitment to take on islamic state even if that meant working with the big bad russians doing it flynn took a hit  talking to them trump was supposed to have revealed state secrets doing it but now the existential threat rears its ugly head and russia is not  it   idiotic newspaper idiotic democrats [19, 169, 180, 215, 275, 349, 370, 389]
['change', 'gay', 'gender', 'guide', 'lesbian', 'neutral', 'transgender', 'want'] thats not good enough for the left they want to change the name to gender neutral transgender gay and lesbian guides [4, 35, 118]
[] sorry old mani knew you were a drunk during  the  years in alaska we have grossed  million dollars wwwthehealingcentermedicalclinicorg [13, 14, 15, 16,

['aircraft', 'bud', 'cash', 'didnt', 'enabler', 'enablers', 'eskom', 'fee', 'floating', 'good', 'idiot', 'imagine', 'job', 'le', 'lease', 'loan', 'mind', 'say', 'spring', 'thing', 'comment', 'day', 'hawk', 'mind', 'paragraph', 'really', 'stick', 'away', 'banana', 'doubt', 'drunken', 'prof', 'republic', 'sa', 'step'] funny how the rot at saa seems to be largely gupta free or have i missed it i can only imagine its because there is less cash floating around than say eskom or transnet or the idiot enabler dudu didnt do a good enough job and things were nipped in the bud the aircraft lease and loan enablers fee spring to mind   the paragraph that really sticks in my mind is same soles comment about what the npa and hawks should have done from day  it proves beyond a doubt that sa is but one drunken step away from a  banana republic [56, 76, 77, 171, 194, 199, 262, 318, 319, 379, 445, 446, 533]
['cut', 'dont', 'guy', 'idiot', 'leave', 'loose', 'somebody'] amarcas this guys where cut loose b

['bullshit', 'conversation', 'injected', 'instead', 'like', 'meaningless', 'typical', 'wa'] if that was only the case instead like a typical trumpnut all you injected into the conversation was meaningless bullshit [25, 34, 59, 124]
['america', 'idiotic', 'voter'] exactly in  and again in  americas voters could not have been more idiotic [7, 12, 13, 14, 15, 16, 31, 32, 33, 34, 35, 44, 86]
['clown', 'country', 'dems', 'fool', 'head', 'like', 'look', 'making', 'oh', 'trumpnuts', 'voted'] oh no kabooby its not the dems that are making us all look like fools its the trumpnuts who voted for this clown to head our country [5, 14, 18, 43, 73, 74, 78, 139]
['idiot', 'youre'] youre an idiot [3, 15, 16, 17]
['beat', 'crap', 'kick', 'think'] it will likely be k for justin and kathleen actually  but no matter   beats the crap out of the  they will get from cpp  the other thing is that the balance in investments can still grow even as you draw down  and yes there is a minimum withdrawl each year  bu

['ignorant', 'lying', 'way'] allie in  only  million came from general funds   million came from the so called ceded land revenues and  million from their endowment whose original corpus was revenues due from so called ceded lands  you are either ignorant or lying either way paa ka waha [0, 6, 7, 12, 13, 14, 15, 22, 23, 24, 25, 58, 61, 62, 63, 64, 65, 118, 124, 125, 126, 127, 128, 159, 225, 226, 261, 273, 277, 287]
['comment', 'forth', 'garbage', 'like', 'spew', 'totally', 'unbelievable', 'unsupported'] edwin you are a joke  first you make claims about longevity that have long ago been debunked because yiu couldnt be bothered to read the whole story  you change your tune and say you cant judge reliabilty by high milegae you keep changing the factors to try and perpetuate your lies before you were comparing high mileage to longevity and now you want to change that   totally unbelievable like most of the garbage and unsupported comments you spew forth [5, 21, 113, 151, 224, 287, 372, 461

['legislate', 'stupidity', 'want'] why would you want to legislate stupidity [41]
['ball', 'freaking', 'run', 'denver', 'just', 'stupid'] the denver donkeys are back  the special teams coach mccoy and the head coach should lose their jobs  you put a guard in a tackle and cant figure out how to give the guy some help  we never seem to do this  defense was good but still cant cover backs or tes  special teams was less than special  offense with the great pretender at qb is terrible  he must be the best practice player in the world but the worst when it comes to game time  mcnown was pretty bad too  run the freaking ball  sd had the worst d against the run  denver is just being stupid [27, 53, 102, 140, 183, 210, 243, 263, 265, 303, 356, 448, 476, 500, 537, 567]
['incredibly', 'people', 'stupid'] she opened the front door and fired a round because she was angry it cost a family a loved one i dont know  years seems too short in some ways how are people this incredibly stupid [43, 66, 96, 1

In [20]:
test_df_preprocessed.head(50)

,spans,text,toxicity,toxic_words,original_text,sentences,diff,toxicity_sentence,predicted_span
0,"[15, 16, 17, 18, 19, 27, 28, 29, 30, 31]",because hes a moron and a bigot. its not any m...,1,"[moron, bigot]",Because he's a moron and a bigot. It's not any...,"[because hes a moron and a bigot., its not any...","[10, 36]","[1.0, 0.0]","[27, 28, 29, 30, 31, 32, 15, 16, 17, 18, 19]"
1,"[29, 30, 31, 32, 33, 34]",how about we stop protecting idiots and let na...,1,[idiots],How about we stop protecting idiots and let na...,[how about we stop protecting idiots and let n...,[],"[1.0, 0.0]","[51, 52, 53, 60, 61, 62, 63, 64, 65, 74, 75, 7..."
2,"[166, 167, 168, 169, 170, 171]","if people were smart, they would boycott th...",1,[idiots],"If people were smart, they would Boycott th...","[if people were smart, they would boycott t...",[],[1.0],[]
3,"[87, 88, 89, 90, 91, 92]",trump claimed that russia will never invade th...,1,[stupid],Trump Claimed that Russia will never invade th...,[trump claimed that russia will never invade t...,[],[1.0],[]
4,[],as long as your willing to pay a lot more for ...,0,[],As long as your willing to pay a lot more for ...,[as long as your willing to pay a lot more for...,[148],"[0.0, 0.0, 0.0]","[188, 189, 190, 191, 192, 193, 194, 195, 196, ..."
5,"[8, 9, 10, 11, 12]",only an idiot would use and believe anything t...,1,[idiot],Only an idiot would use and believe anything t...,[only an idiot would use and believe anything ...,[],[1.0],"[28, 29, 30, 31, 32, 33, 34, 8, 9, 10, 11, 12,..."
6,"[265, 266, 267, 268, 269, 270, 271, 272, 273, ...",thanks a lot douchebag. youre the reason the p...,1,[o try to turn salem into some kind of new-stu...,Thanks a lot douchebag. You're the reason the ...,"[thanks a lot douchebag., youre the reason the...","[27, 467]","[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0]","[13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 9, 10..."
7,[],kick all the non human criminal illegals out o...,0,[],kick all the non human criminal illegals out o...,[kick all the non human criminal illegals out ...,[],[0.0],"[23, 24, 25, 26, 27, 28, 29, 30, 73, 74, 75, 7..."
8,"[38, 39, 40, 41, 42, 43]",because driving under ontario laws is stupid e...,1,[stupid],Because driving under Ontario laws is stupid e...,[because driving under ontario laws is stupid ...,[],[1.0],"[8, 9, 10, 11, 12, 13, 14, 30, 31, 32, 22, 23,..."
9,"[277, 278, 279, 280, 281, 282, 283, 284, 285, ...",youre wrong. the delay between retirement and...,1,[dont make ignorant statements],You're wrong. The delay between retirement an...,"[youre wrong., the delay between retirement an...","[3, 264, 280]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]","[277, 278, 279, 280, 281, 288, 289, 290, 291, ..."


In [21]:
test_df_preprocessed["Pscore"] = [ 1 if (len(s) == 0 and len(ps) == 0) 
                             else 0 if len(ps) == 0 
                             else len( set(s).intersection(set(ps) ))/ len(set(ps))  for s, ps in zip(test_df_preprocessed["spans"],test_df_preprocessed["predicted_span"]) ]

In [22]:
test_df_preprocessed["Rscore"] = [ 1 if (len(s) == 0 and len(ps) == 0) 
                             else 0 if len(s) == 0 
                             else len( set(s).intersection(set(ps) ))/ len(set(s))  for s, ps in zip(test_df_preprocessed["spans"],test_df_preprocessed["predicted_span"]) ]

In [23]:
test_df_preprocessed["Fscore"] = 2 * test_df_preprocessed["Pscore"] *test_df_preprocessed["Rscore"] /(test_df_preprocessed["Pscore"] + test_df_preprocessed["Rscore"])

In [24]:
F_score= np.mean(test_df_preprocessed["Fscore"])

In [25]:
F_score

0.4234201722499819